In [2]:
import tensorflow as tf
print(tf.test.is_gpu_available())

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
True


In [3]:
layer = tf.keras.layers.Dense(100)
layer = tf.keras.layers.Dense(10, input_shape = (None, 5))
#y有关于keras的全部可用功能看到https://tensorflow.google.cn/api_docs/python/tf/keras/layers

In [4]:
layer(tf.zeros([10,5]))

<tf.Tensor: shape=(10, 10), dtype=float32, numpy=
array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]], dtype=float32)>

In [5]:
layer.variables

[<tf.Variable 'dense_1/kernel:0' shape=(5, 10) dtype=float32, numpy=
 array([[ 0.11527622, -0.48693377, -0.5839218 ,  0.51580745, -0.23092273,
         -0.37315896, -0.44899142,  0.56136006,  0.150172  ,  0.5493092 ],
        [-0.4245417 , -0.5442909 , -0.05779731,  0.5966503 ,  0.06543297,
         -0.34715357, -0.17304471,  0.33078587,  0.51651365,  0.138547  ],
        [-0.47808862, -0.061647  , -0.5737171 ,  0.17473555,  0.14625072,
         -0.54222375,  0.28609556,  0.08838367,  0.17017883,  0.39209932],
        [-0.01161784,  0.49357027, -0.36919895,  0.22138137,  0.31170666,
         -0.24094132,  0.06301039,  0.02464044,  0.47321695, -0.2549142 ],
        [-0.04676336,  0.04095614,  0.21719307,  0.2788465 ,  0.13980716,
         -0.11309004, -0.30194154, -0.28280234, -0.37080544, -0.3699969 ]],
       dtype=float32)>,
 <tf.Variable 'dense_1/bias:0' shape=(10,) dtype=float32, numpy=array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)>]

In [6]:
#write the custom class using keras
class MyDenseLayer(tf.keras.layers.Layer):
    #where you do the input independent initialization
    def __init__(self, num_outputs):
        super(MyDenseLayer, self).__init__()
        self.num_outputs = num_outputs
    #where you know the shapes of the input tensors and can do the rest of the initialization
    def build(self, input_shape):
        self.kernel = self.add_weight("kernel",
                                     shape = [int(input_shape[-1]),
                                            self.num_outputs])  
                                        
    #call, where you do the forward computation                                 
    def call(self, inputs):
        return tf.matmul(inputs, self.kernel)

layer = MyDenseLayer(10)

In [7]:
_ = layer(tf.zeros([10, 5]))#calling the layer '.builds' it
print([var.name for var in layer.trainable_variables])

['my_dense_layer/kernel:0']


In [13]:
#typically yu=ou inherit from keras.model when you need the model methods like model.fit, model.evaluate and model.save

class ResnetIdentityBlock(tf.keras.Model):
    def __init__(self, kernel_size, filters):
        super(ResnetIdentityBlock, self).__init__(name='')
        filters1, filters2, filters3 = filters
        
        self.conv2a = tf.keras.layers.Conv2D(filters1, (1,1))
        self.bn2a = tf.keras.layers.BatchNormalization()
        self.conv2b = tf.keras.layers.Conv2D(filters2, kernel_size, padding='same')
        self.bn2b = tf.keras.layers.BatchNormalization()
        self.conv2c = tf.keras.layers.Conv2D(filters3, (1,1))
        self.bn2c = tf.keras.layers.BatchNormalization()
        
    def call(self, input_tensor, training = False):
        x = self.conv2a(input_tensor)
        x = self.bn2a(x, training=training)
        x = tf.nn.relu(x)
        
        x = self.conv2b(x)
        x = self.bn2b(x, training = training)
        x = tf.nn.relu(x)
        
        x = self.conv2c(x)
        x = self.bn2c(x, training = training)
        
        x += input_tensor
        return tf.nn.relu(x)
    
block = ResnetIdentityBlock(1, [1,2,3])



In [14]:
_ = block(tf.zeros([1,2,3,3]))
block.layers

In [15]:
len(block.variables)

18

In [16]:
block.summary()

Model: "resnet_identity_block_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            multiple                  4         
_________________________________________________________________
batch_normalization_6 (Batch multiple                  4         
_________________________________________________________________
conv2d_7 (Conv2D)            multiple                  4         
_________________________________________________________________
batch_normalization_7 (Batch multiple                  8         
_________________________________________________________________
conv2d_8 (Conv2D)            multiple                  9         
_________________________________________________________________
batch_normalization_8 (Batch multiple                  12        
Total params: 41
Trainable params: 29
Non-trainable params: 12
______________________________________________

In [18]:
#another simple method to construct model

my_seq = tf.keras.Sequential([tf.keras.layers.Conv2D(1,(1,1),
                                                    input_shape=(
                                                    None, None, 3)),
                             tf.keras.layers.BatchNormalization(),
                             tf.keras.layers.Conv2D(2,1,
                                                   padding='same'),
                             tf.keras.layers.BatchNormalization(),
                             tf.keras.layers.Conv2D(3,(1,1)),
                             tf.keras.layers.BatchNormalization()])
my_seq(tf.zeros([1,2,3,3]))

<tf.Tensor: shape=(1, 2, 3, 3), dtype=float32, numpy=
array([[[[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],

        [[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]]]], dtype=float32)>

In [19]:
my_seq.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_10 (Conv2D)           (None, None, None, 1)     4         
_________________________________________________________________
batch_normalization_10 (Batc (None, None, None, 1)     4         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, None, None, 2)     4         
_________________________________________________________________
batch_normalization_11 (Batc (None, None, None, 2)     8         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, None, None, 3)     9         
_________________________________________________________________
batch_normalization_12 (Batc (None, None, None, 3)     12        
Total params: 41
Trainable params: 29
Non-trainable params: 12
___________________________________________________________